# Question 2: Dealing with Messy and Missing Data

CS 5304 - Data Science in the Wild, Assignment 2

**Author**: Yufan Zhang (yz2894)


In [78]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import ttest_ind, chi2_contingency

In [32]:
# Read in the individual data
individual_data = pd.read_pickle("data/Extrasensory_individual_data.p")

print("Individual data shape:", individual_data.shape)

individual_data.head()

Individual data shape: (60, 7)


,uuid,age,gender,system,hours_in_study,perceived_average_screen_time,actual_average_screen_time
0,3600D531-0C55-44A7-AE95-A7A38519464E,24,male,Android,86,4.3,5.18
1,59EEFAE0-DEB0-4FFF-9250-54D2A03D0CF2,31,female,Android,125,4.2,2.31
2,CF722AA9-2533-4E51-9FEB-9EAC84EE9AAC,37,male,iOS,60,3.9,-1.00
3,5152A2DF-FAF3-4BA8-9CA9-E66B32671A53,22,male,iOS,110,-1.0,4.75
4,136562B6-95B2-483D-88DC-065F28409FD2,19,male,Android,103,1.1,1.55


In [33]:
# Read in the sensor data
sensor_data = pd.read_pickle("data/Extrasensory_sensor_data.p")

print("Sensor data count:", len(sensor_data))

print(
    "Shape of the first sensor data:",
    sensor_data["3600D531-0C55-44A7-AE95-A7A38519464E"].shape,
)

sensor_data["3600D531-0C55-44A7-AE95-A7A38519464E"].head()

Sensor data count: 60
Shape of the first sensor data: (5203, 10)


,location:raw_latitude,location:raw_longitude,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,discrete:app_state:is_active,discrete:app_state:is_inactive,discrete:app_state:is_background,discrete:app_state:missing,lf_measurements:battery_level
0,32.882483,-117.234601,0.022972,-0.002678,-1.002311,0.0,1.0,0.0,0.0,0.49
1,32.882480,-117.234595,0.021403,-0.002784,-1.001535,0.0,0.0,1.0,0.0,0.50
2,32.882482,-117.234587,0.021201,-0.004036,-1.000577,0.0,1.0,0.0,0.0,0.56
3,32.882482,-117.234587,0.021530,-0.004327,-0.998347,1.0,0.0,0.0,0.0,0.56
4,32.882483,-117.234582,0.021198,-0.004350,-1.004198,0.0,1.0,0.0,0.0,0.57


### Case 1: Actual screen time


In [34]:
## Case 1 Problem A code (and optional graph)
# Plot the histogram of the actual average screen time with plotly express
fig = px.histogram(
    individual_data,
    x="actual_average_screen_time",
    nbins=50,
    title="Actual Average Screen Time",
)
fig.update_layout(width=800, height=500)
fig.show()

# Count the number of missing values (-1) in actual_average_screen_time in the individual data
missing_values = individual_data["actual_average_screen_time"].value_counts()[-1]

# Print the number of missing values
print(missing_values)

4


#### Writeup Answer to Problem A: 

**How are missing values represented for this feature?**

**Answer**: Missing values are represented as "-1" in the dataset.

In [35]:
## Case 1 Problem B code and graph
# Ignore the missing values for now (i.e. mute them or temporarily remove them, so that they don’t affect this analysis).
individual_data_temp = individual_data[
    individual_data["actual_average_screen_time"] != -1
]

# Provide a histogram to illustrate this distribution of actual_average_screen_time
fig = px.histogram(
    individual_data_temp,
    x="actual_average_screen_time",
    nbins=30,
    title="Distribution of actual average screen time",
)

# Resize the plot
fig.update_layout(width=800, height=500)
fig.show()

In [36]:
# Calculate the high and low quantiles for detecting outliers
# An outlier is defined as being any point of data that lies over 1.5 IQRs below the first quartile (Q1) or above the third quartile (Q3)in a data set.
# High = (Q3) + 1.5 IQR
# Low = (Q1) – 1.5 IQR

# Calculate the first and third quartiles
Q1 = individual_data_temp["actual_average_screen_time"].quantile(0.25)
Q3 = individual_data_temp["actual_average_screen_time"].quantile(0.75)

# Calculate the interquartile range
IQR = Q3 - Q1

# Calculate the high and low quantiles
high = Q3 + 1.5 * IQR
low = Q1 - 1.5 * IQR

# Print the high and low quantiles
print("High quantile:\t", high)
print("Low quantile:\t", low)

# Count the number of outliers
outliers = individual_data_temp[
    (individual_data_temp["actual_average_screen_time"] > high)
    | (individual_data_temp["actual_average_screen_time"] < low)
]

print("Number of outliers:", len(outliers))
outliers

High quantile:	 6.637499999999999
Low quantile:	 1.0975000000000001
Number of outliers: 2


,uuid,age,gender,system,hours_in_study,perceived_average_screen_time,actual_average_screen_time
15,1155FF54-63D3-4AB2-9863-8385D0BD0A13,23,male,Android,44,-1.0,10.78
44,33A85C34-CFE4-4732-9E73-0A7AC861B27A,29,male,Android,102,-1.0,11.63


In [37]:
# Check skewness of the actual_average_screen_time
skewness = individual_data_temp["actual_average_screen_time"].skew()

# Print the skewness
print("Skewness:", skewness)

# Interpretation
if skewness < -1:
    print("The distribution is highly skewed to the left.")
elif -1 <= skewness < -0.5:
    print("The distribution is moderately skewed to the left.")
elif -0.5 <= skewness <= 0.5:
    print("The distribution is approximately symmetric.")
elif 0.5 < skewness <= 1:
    print("The distribution is moderately skewed to the right.")
else:
    print("The distribution is highly skewed to the right.")

Skewness: 2.4639261915800215
The distribution is highly skewed to the right.


#### Writeup Answer to Problem B: 

**Does it have outliers? If so, how many?**

**Answer**: Yes. It has 2 outliers.

**Is it skewed? If so, is it left skewed or right skewed? What’s the skewness?**

**Answer**: It is right skewed. The skewness is 2.46.


In [47]:
## Case 1 Problem C code and graph
individual_data_temp = individual_data[
    individual_data["actual_average_screen_time"] != -1
]

# 1) Fill the missing value with the median
median = individual_data_temp["actual_average_screen_time"].median()
print("Median:\t", median)
individual_data["actual_average_screen_time_filled_na_with_median"] = individual_data[
    "actual_average_screen_time"
].replace(-1, median)

# 2) Fill the missing value with the mean
mean = individual_data_temp["actual_average_screen_time"].mean()
print("Mean:\t", mean)
individual_data["actual_average_screen_time_filled_na_with_mean"] = individual_data[
    "actual_average_screen_time"
].replace(-1, mean)


# 3) Fill the missing value with random values
# Generate random values from the set of observed (non-missing) values
def sample_one_from_actual_average_screen_time():
    random_value = individual_data_temp["actual_average_screen_time"].sample().values[0]
    print("Random value:\t", random_value)
    return random_value


individual_data["actual_average_screen_time_filled_na_with_random"] = individual_data[
    "actual_average_screen_time"
].apply(lambda x: sample_one_from_actual_average_screen_time() if x == -1 else x)

Median:	 3.715
Mean:	 3.9728571428571433
Random value:	 11.63
Random value:	 1.76
Random value:	 11.63
Random value:	 4.02


In [48]:
# Plotting
fig = go.Figure()

# Original distribution
fig.add_trace(
    go.Histogram(
        x=individual_data_temp["actual_average_screen_time"],
        name="Original",
        opacity=0.5,
        marker_color="blue",
        nbinsx=30,
    )
)

# Median filled
fig.add_trace(
    go.Histogram(
        x=individual_data["actual_average_screen_time_filled_na_with_median"],
        name="Filled with Median",
        opacity=0.5,
        marker_color='yellow',
        nbinsx=30,
    )
)

# Mean filled
fig.add_trace(
    go.Histogram(
        x=individual_data["actual_average_screen_time_filled_na_with_mean"],
        name="Filled with Mean",
        opacity=0.5,
        marker_color='green',
        nbinsx=30,
    )
)

# Random filled
fig.add_trace(
    go.Histogram(
        x=individual_data["actual_average_screen_time_filled_na_with_random"],
        name="Filled with Random",
        opacity=0.5,
        marker_color='red',
        nbinsx=30,
    )
)

# Update the layout
fig.update_layout(
    width=800,
    height=500,
    barmode="overlay",
    title_text="Distribution of Actual Average Screen Time with Different NA Filling Techniques",
)

fig.show()

#### Writeup Answer to Problem C: 

**How did you choose the random value from method 3)?**

**Answer**: Assuming the missingness is completely random, I chose a random value from the set of observed (non-missing) values in the same column (i.e., `actual_average_screen_time`).

**How do the distributions look like after you implement the three filling methods? (Compare them)**

**Answer**: As the mean (3.973) and median (3.715) values are so close to each other that the the filled values actually fall in the same bin, the distributions after filling the missing values using method 1 (filling with the mean) and method 2 (filling with the median) are the same. For method 3 (filling with a random value), the distribution also remains relatively unchanged since this method is akin to a resampling technique, where we're essentially replicating the observed data to fill in gaps caused by missing values.

In [76]:
## Case 1 Problem D code and graph
# Simulated population distribution based on research
np.random.seed(0)
population_distribution = np.random.normal(3.85, 1.25, len(individual_data))

print("Population distribution shape:", population_distribution.shape)

# Plot the population distribution
fig = px.histogram(
    x=population_distribution,
    title="Simulated Population Distribution",
    nbins=30,
)
fig.update_layout(width=800, height=500)

fig.show()

Population distribution shape: (60,)


In [77]:
# Perform t-tests
ttest_median = ttest_ind(individual_data["actual_average_screen_time_filled_na_with_median"], population_distribution)
ttest_mean = ttest_ind(individual_data["actual_average_screen_time_filled_na_with_mean"], population_distribution)
ttest_random = ttest_ind(individual_data["actual_average_screen_time_filled_na_with_random"], population_distribution)

ttest_median.pvalue, ttest_mean.pvalue, ttest_random.pvalue

(0.9719606601354556, 0.9230094384428134, 0.4625514594221577)

#### Answer to Problem D: 

**Report the three p-values. Which one of the filling methods reconstruct this feature to be closest to the research distribution? Why do you think this is the case?**

**Answer**: The p-values for the three filling methods are as follows:

- For the median filled distribution: **p-value = 0.9720**
- For the mean filled distribution: **p-value = 0.9230**
- For the random value filled distribution: **p-value = 0.4625**

Since all p-values are greater than 0.05, none of the methods show a statistically significant difference from the research distribution. This suggests that all methods are somewhat effective in reconstructing the feature to be close to the research distribution. However, the method that filled missing values with **median value** has the highest p-value (0.9720), indicating the least statistical difference from the population distribution and hence, it is the closest to the research distribution among the three methods.


### Case 2: Perceived average screen time

In [80]:
## Case 2 Problem A code and histogram
# Plot the histogram of the actual average screen time with plotly express
fig = px.histogram(
    individual_data,
    x="perceived_average_screen_time",
    nbins=50,
    title="Perceived Average Screen Time",
)
fig.update_layout(width=800, height=500)
fig.show()

# Count the number of missing values (-1) in actual_average_screen_time in the individual data
missing_values = individual_data["perceived_average_screen_time"].value_counts()[-1]

# Print the number of missing values
print(f"Missing value count:\t{missing_values}")

Missing value count:	7


In [81]:
# Calculate the high and low quantiles for detecting outliers
# An outlier is defined as being any point of data that lies over 1.5 IQRs below the first quartile (Q1) or above the third quartile (Q3)in a data set.
# High = (Q3) + 1.5 IQR
# Low = (Q1) – 1.5 IQR

individual_data_temp = individual_data[
    individual_data["perceived_average_screen_time"] != -1
]

# Calculate the first and third quartiles
Q1 = individual_data_temp["perceived_average_screen_time"].quantile(0.25)
Q3 = individual_data_temp["perceived_average_screen_time"].quantile(0.75)

# Calculate the interquartile range
IQR = Q3 - Q1

# Calculate the high and low quantiles
high = Q3 + 1.5 * IQR
low = Q1 - 1.5 * IQR

# Print the high and low quantiles
print("High quantile:\t", high)
print("Low quantile:\t", low)

# Count the number of outliers
outliers = individual_data_temp[
    (individual_data_temp["perceived_average_screen_time"] > high)
    | (individual_data_temp["perceived_average_screen_time"] < low)
]

print("Number of outliers:", len(outliers))
outliers

High quantile:	 5.8500000000000005
Low quantile:	 1.45
Number of outliers: 2


,uuid,age,gender,system,hours_in_study,perceived_average_screen_time,actual_average_screen_time,actual_average_screen_time_filled_na_with_median,actual_average_screen_time_filled_na_with_mean,actual_average_screen_time_filled_na_with_random
4,136562B6-95B2-483D-88DC-065F28409FD2,19,male,Android,103,1.1,1.55,1.55,1.55,1.55
32,5EF64122-B513-46AE-BCF1-E62AAC285D2C,36,female,iOS,65,6.0,6.06,6.06,6.06,6.06


In [82]:
# Check skewness of the actual_average_screen_time
skewness = individual_data_temp["perceived_average_screen_time"].skew()

# Print the skewness
print("Skewness:", skewness)

# Interpretation
if skewness < -1:
    print("The distribution is highly skewed to the left.")
elif -1 <= skewness < 0:
    print("The distribution is moderately skewed to the left.")
elif 0 <= skewness <= 1:
    print("The distribution is moderately skewed to the right.")
else:
    print("The distribution is highly skewed to the right.")

Skewness: -0.21439363473731404
The distribution is moderately skewed to the left.


#### Writeup Answer to Problem A: 

**Does it have outliers? If so, how many?**

**Answer**: Yes. It has 2 outliers.

**Is it skewed? If so, is it left skewed or right skewed? What’s the skewness?**

**Answer**: The distribution is moderately skewed to the left.

In [15]:
## Case 2 Problem B code
# Calculate mean and standard deviation of actual_average_screen_time
mean_screen_time = individual_data['actual_average_screen_time'].mean()
std_screen_time = individual_data['actual_average_screen_time'].std()

print("Mean:", mean_screen_time)
print("Standard Deviation:", std_screen_time)

# Calculate the threshold for intense phone use
intense_use_threshold = mean_screen_time + std_screen_time

# Identify intense phone users
intense_users = individual_data[individual_data['actual_average_screen_time'] >= intense_use_threshold]

# Get the number of intense phone users
number_of_intense_users = intense_users.shape[0]

# Print the number of intense phone users
print("Number of intense phone users:", number_of_intense_users)

intense_users

Mean: 3.6413333333333338
Standard Deviation: 2.109741302051491
Number of intense phone users: 4


,uuid,age,gender,system,hours_in_study,perceived_average_screen_time,actual_average_screen_time,actual_average_screen_time_filled_na_with_median,actual_average_screen_time_filled_na_with_mean,actual_average_screen_time_filled_na_with_random
15,1155FF54-63D3-4AB2-9863-8385D0BD0A13,23,male,Android,44,-1.0,10.78,10.78,10.78,10.78
32,5EF64122-B513-46AE-BCF1-E62AAC285D2C,36,female,iOS,65,6.0,6.06,6.06,6.06,6.06
44,33A85C34-CFE4-4732-9E73-0A7AC861B27A,29,male,Android,102,-1.0,11.63,11.63,11.63,11.63
45,5119D0F8-FCA8-4184-A4EB-19421A40DE0D,25,female,Android,110,3.2,5.94,5.94,5.94,5.94


#### Writeup Answer to Problem B: 

**How many of them are intense phone users?**

Answer: 4

In [16]:
## Case 2 Problem C code and graph
# Step 1: Filter out users with missing actual screen time
filtered_data = individual_data[individual_data['actual_average_screen_time'] != -1]

# Step 2: Generate boolean arrays
# A) Whether perceived_average_screen_time is missing
missing_perceived = filtered_data['perceived_average_screen_time'] == -1

# B) Intense phone users based on actual_average_screen_time
mean_screen_time = filtered_data['actual_average_screen_time'].mean()
std_screen_time = filtered_data['actual_average_screen_time'].std()
intense_threshold = mean_screen_time + std_screen_time
is_intense_user = filtered_data['actual_average_screen_time'] >= intense_threshold

# Step 3: Perform Chi-square test
# Constructing a contingency table
contingency_table = pd.crosstab(missing_perceived, is_intense_user)

chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square test p-value: {p_value}")

# Interpretation based on p-value
if p_value < 0.05:
    print("There is a significant relationship between missing perceived_average_screen_time and being an intense phone user.")
    print("This suggests the data is MNAR (missing not at random).")
else:
    print("There is no significant relationship between missing perceived_average_screen_time and being an intense phone user.")
    print("This suggests the data could be MAR (missing at random).")

Chi-square test p-value: 0.11666446478102341
There is no significant relationship between missing perceived_average_screen_time and being an intense phone user.
This suggests the data could be MAR (missing at random).


#### Writeup Answer to Problem B: 

**What is the p-value? Do you think they are correlated? What does this mean? Do you think this feature is MAR or MNAR?**

Answer: The p-value is 0.11666. There is no significant relationship between missing perceived_average_screen_time and being an intense phone user. Thus, the missingness of this feature is likely to be Missing At Random (MAR).

### Case 3: Location

In [17]:
## Case 3 Problem A code (graph)
sensor_data["3600D531-0C55-44A7-AE95-A7A38519464E"].head()

,location:raw_latitude,location:raw_longitude,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,discrete:app_state:is_active,discrete:app_state:is_inactive,discrete:app_state:is_background,discrete:app_state:missing,lf_measurements:battery_level
0,32.882483,-117.234601,0.022972,-0.002678,-1.002311,0.0,1.0,0.0,0.0,0.49
1,32.882480,-117.234595,0.021403,-0.002784,-1.001535,0.0,0.0,1.0,0.0,0.50
2,32.882482,-117.234587,0.021201,-0.004036,-1.000577,0.0,1.0,0.0,0.0,0.56
3,32.882482,-117.234587,0.021530,-0.004327,-0.998347,1.0,0.0,0.0,0.0,0.56
4,32.882483,-117.234582,0.021198,-0.004350,-1.004198,0.0,1.0,0.0,0.0,0.57


In [18]:
sensor_data["3600D531-0C55-44A7-AE95-A7A38519464E"].columns

Index(['location:raw_latitude', 'location:raw_longitude', 'raw_acc:3d:mean_x',
       'raw_acc:3d:mean_y', 'raw_acc:3d:mean_z',
       'discrete:app_state:is_active', 'discrete:app_state:is_inactive',
       'discrete:app_state:is_background', 'discrete:app_state:missing',
       'lf_measurements:battery_level'],
      dtype='object')

In [85]:
## Case 3 Problem A code (graph)
# Analyzing data to identify users turning off location services when battery is below 20%
def check_below_20_and_missing_location(x):
    return x["lf_measurements:battery_level"] < 0.2 and pd.isnull(x["location:raw_latitude"])


def identify_users(sensor_data):
    users_behavior = {}
    for uuid, df in sensor_data.items():
        # Filter for records where battery level is below 20%
        below_20_and_missing_location = df.apply(check_below_20_and_missing_location, axis=1)
        
        # Filter the behavior 2 users
        behavior_2_df = df[below_20_and_missing_location]
        
        totel_behavior_2_count = behavior_2_df.shape[0]
        total_below_20_count = df[df["lf_measurements:battery_level"] < 0.2].shape[0]
        
        # Check if a significant portion of low battery records have missing location data
        if totel_behavior_2_count > 0 and totel_behavior_2_count == total_below_20_count:
            # Report UUID and how many minutes of data are lost
            users_behavior[uuid] = totel_behavior_2_count

    return users_behavior


users_with_consistent_behavior = identify_users(sensor_data)

print("Number of users with consistent behavior:", len(users_with_consistent_behavior))

users_with_consistent_behavior

Number of users with consistent behavior: 5


{'098A72A5-E3E5-4F54-A152-BBDA0DF7B694': 415,
 'CDA3BBF7-6631-45E8-85BA-EEB416B32A3C': 74,
 '96A358A0-FFF2-4239-B93E-C7425B901B47': 277,
 'B09E373F-8A54-44C8-895B-0039390B859F': 369,
 'B7F9D634-263E-4A97-87F9-6FFB4DDCB36C': 176}

#### Writeup Answer to Problem A: 

**Explanation of implementation**

**Answer**: Firstly, I employ the function, `check_below_20_and_missing_location`, which scrutinizes each data record to determine whether the battery level is below 20% and the location data (`location:raw_latitude`) is missing. After isolating records that indicate potential behavior 2, I compare the count of these records against the total count of records with battery levels below 20% for each user. If all of low-battery records for a user lacks location data, it suggests that the user likely turns off location services to conserve battery power. 

In [86]:
## Case 3 Problem B code and graph
sensor_df_B = sensor_data["F50235E0-DD67-4F2A-B00B-1F31ADA998B9"]

# Count the null values in location:raw_longitude
null_count = sensor_df_B["location:raw_longitude"].isnull().sum()

print("Number of missing values in location:raw_longitude:", null_count)

sensor_df_B.head()

Number of missing values in location:raw_longitude: 156


,location:raw_latitude,location:raw_longitude,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,discrete:app_state:is_active,discrete:app_state:is_inactive,discrete:app_state:is_background,discrete:app_state:missing,lf_measurements:battery_level
0,32.882301,-117.234683,-0.018860,0.037530,1.038409,0.0,0.0,0.0,1.0,0.80
1,32.882297,-117.234704,0.049835,0.092630,0.983184,0.0,0.0,0.0,1.0,0.78
2,32.882288,-117.234700,0.015455,0.012330,1.043454,0.0,0.0,0.0,1.0,0.78
3,32.882315,-117.234685,0.014795,0.010820,1.048484,0.0,0.0,0.0,1.0,0.77
4,32.882304,-117.234690,0.014965,0.011975,1.044709,0.0,0.0,0.0,1.0,0.77


In [91]:
# Assuming sensor_df_B is a given dataframe, we'll simulate it for the example.
# Forward filling
sensor_df_B_ffill = sensor_df_B.fillna(method='ffill')

# Backward filling
sensor_df_B_bfill = sensor_df_B.fillna(method='bfill')

# Linear interpolation
sensor_df_B_interpolated = sensor_df_B.interpolate(method='linear')

# Create a Plotly figure with 4 traces
fig = go.Figure()

# Original data
fig.add_trace(go.Scatter(x=sensor_df_B.index, y=sensor_df_B['location:raw_latitude'], mode='lines+markers', name='Original'))

# Forward filled data
fig.add_trace(go.Scatter(x=sensor_df_B_ffill.index, y=sensor_df_B_ffill['location:raw_latitude'], mode='lines+markers', name='Forward Fill'))

# Backward filled data
fig.add_trace(go.Scatter(x=sensor_df_B_bfill.index, y=sensor_df_B_bfill['location:raw_latitude'], mode='lines+markers', name='Backward Fill'))

# Linearly interpolated data
fig.add_trace(go.Scatter(x=sensor_df_B_interpolated.index, y=sensor_df_B_interpolated['location:raw_latitude'], mode='lines+markers', name='Linear Interpolation'))

fig.update_layout(title='Comparison of Filling Methods for Missing Data', xaxis_title='Minute', yaxis_title='Location: Raw Latitude')

fig.update_layout(width=1000, height=600)
fig.show()


/var/folders/_6/vc6bwqjd62501rmy4hvwykhw0000gn/T/ipykernel_50936/269646697.py:3: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/_6/vc6bwqjd62501rmy4hvwykhw0000gn/T/ipykernel_50936/269646697.py:6: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



#### Writeup Answer to Problem B: 

**Compare the 4 traces. What do you see? If you were to use this dataset for further analysis, which filling method will you choose?**

**Answer**: The 4 traces show that the distribution of the feature `location:raw_latitude` is pretty much the same for the 3 filling methods. Therefore, using any of the filling methods will not significantly affect the distribution of the feature, at least for this dataset.